In [15]:
import pandas as pd
import numpy as np
#from random import randint
import matplotlib.pyplot as plt
#import matplotlib.ticker as mtick
import warnings
import seaborn as sns

warnings.filterwarnings('ignore')

ds_or = pd.read_csv('dataset/ds_or_gate.csv')
ds_t = pd.read_csv('dataset/ds_t.csv')
ds_iris = pd.read_csv('dataset/iris.csv')
ds_t.head()

,X,Y,Class
0,0.3,0.70,1
1,-0.6,0.30,0
2,-0.1,-0.80,0
3,0.1,-0.45,1


In [2]:
# Calculate of neuron with sums
# W(T)*X = u
# x0*w0 + x1*w1 + x2*w2 ... + xp*wp = u
# Returned:
# S = { u >= 0 -> Y = 1 | u < 0 -> Y = 0 }
def calculate_neuron(X, W):
    u = W.dot(X.T) # It makes X*W(T)
    y = 1 if u >= 0 else 0 # If u > 0 then y = 1 if not, y = 0
    return y

def calculate_error(Y, Expected):
    # e = d - y
    error = Expected - Y
    return error

def calculate_new_w(X, Y, W, Expected, rate):
    # 𝒘(𝑡 + 1) = 𝒘 (𝑡) + 𝜂*𝑒(𝑡)*𝒙(𝑡)
    w_new = W + rate * calculate_error(Y, Expected) * X
    return w_new


In [111]:
def one_hot_encoding(classes):
    cl_onehot = np.zeros((len(classes),len(classes)),dtype=int)
    np.fill_diagonal(cl_onehot,1)
    r = [(classes[i], cl) for i, cl in enumerate(cl_onehot)]
    return r

def encode_expected(expected, encoded_class):
    return [ list(filter(lambda e: e[0] == x, encoded_class))[0][1] for x in expected ]

# Perceptron Simples
def mount_neural_network(ds, wInit, rate):
    #W = np.array([wInit])
    #w_ = []
    
    encoded_class = one_hot_encoding(pd.unique(ds.iloc[:,-1:].values.flatten()))
    expected = ds.iloc[:,-1:].values.flatten()
    expected_encoded = encode_expected(expected, encoded_class)
    
    ds_no_class = ds.iloc[:,:-1]
    ds_no_class.insert(0, 'bias', -1)
    
    W_ALL = np.array([wInit for i in range(0, len(encoded_class))])
    #print(W_ALL)
    #for i,X in enumerate(ds_no_class.values):
    #i = 0
    for i in range(0,50): # TODO: CONDICAO DE PARADA
    #while True:
        print('\n+++++++++++++++++++++++ Rodada N: '+ str(i+1) + ' +++++++++++++++++++++++')
        for e in range(0, len(encoded_class)):
            W = W_ALL[e]
            X = ds_no_class.values[i % len(expected)]
            y = calculate_neuron(np.array([X]), W)
            
            exp = expected_encoded[i % len(expected)][e]
            
            new_W = calculate_new_w(np.array(X), y,W, exp, rate)
            W_ALL[e] = new_W
            
            print('------------- e n: '+str(e)+' --------------')
            print('W:' + str(W))
            print('Entrada (X):' + str(X))
            print('Achado (Y): '+str(y))
            print('Esperado (Expected): '+str(exp))
            print(W_ALL)
            
        
        #print(W)
        #print(w_)
        #if np.trace(W - w_) == 0:
            #break
    return W_ALL


In [112]:
W_ACH = mount_neural_network(ds_t,[1.,2.,3.],0.5)
W_ACH


+++++++++++++++++++++++ Rodada N: 1 +++++++++++++++++++++++
------------- e n: 0 --------------
W:[1. 2. 3.]
Entrada (X):[-1.   0.3  0.7]
Achado (Y): 1
Esperado (Expected): 1
[[1. 2. 3.]
 [1. 2. 3.]]
------------- e n: 1 --------------
W:[1.5  1.85 2.65]
Entrada (X):[-1.   0.3  0.7]
Achado (Y): 1
Esperado (Expected): 0
[[1.   2.   3.  ]
 [1.5  1.85 2.65]]

+++++++++++++++++++++++ Rodada N: 2 +++++++++++++++++++++++
------------- e n: 0 --------------
W:[1. 2. 3.]
Entrada (X):[-1.  -0.6  0.3]
Achado (Y): 0
Esperado (Expected): 0
[[1.   2.   3.  ]
 [1.5  1.85 2.65]]
------------- e n: 1 --------------
W:[1.   1.55 2.8 ]
Entrada (X):[-1.  -0.6  0.3]
Achado (Y): 0
Esperado (Expected): 1
[[1.   2.   3.  ]
 [1.   1.55 2.8 ]]

+++++++++++++++++++++++ Rodada N: 3 +++++++++++++++++++++++
------------- e n: 0 --------------
W:[1. 2. 3.]
Entrada (X):[-1.  -0.1 -0.8]
Achado (Y): 0
Esperado (Expected): 0
[[1.   2.   3.  ]
 [1.   1.55 2.8 ]]
------------- e n: 1 --------------
W:[0.5 1.5 2.4]
Entra

array([[-1.   ,  2.55 ,  1.725],
       [ 0.   , -0.85 , -0.075]])

In [113]:
#WW = np.array([0.  , 2.1 , 2.55 ])

In [114]:
[ calculate_neuron(np.array([-1,0.3,0.7]),Wc) for Wc in W_ACH ]

[1, 0]

In [6]:
import numpy as np
x = np.array([[1,2,3]])
y = np.array([[5,6,7]])


In [7]:
np.trace(x - y)

-4